This notebook is for examining ways of implementing concurrent cell execution.

I'll begin by showing the existing methods, and their weeknesses.

[Take one: using iPython's parallel magic.](http://ipython.org/ipython-doc/stable/parallel/magics.html)

First, run this in a terminal:
```ipcluster start -n 1```

It can't be run here, since ipython doesn't support background processes.

In [16]:
# Start a cluster. We cannot use the ipython shell magic, since this needs to be nonblocking.
import os
os.popen('ipcluster start -n 1')

<open file 'ipcluster start -n 1', mode 'r' at 0x7f1a8ad409c0>

In [18]:
# Import the parallel stuff, and create a client so that the parallel magic will work.
from IPython.parallel import Client
rc = Client()

In [20]:
%px
import time
print(time.time())
time.sleep(5)
print(time.time())

1427238099.64
1427238104.64


Problems:
 - There is no way to interrupt the parallel cell. Control-C does not work.
 - The output appears suddenly at the end. It should appear immediately.
 - It still blocks, despite executing it in another process.

Take two: using Python's parallel magic using the ```--noblock``` command.

In [25]:
%%px --noblock
import time
print(time.time())
time.sleep(5)
print(time.time())

<AsyncResult: execute>

In [26]:
%pxresult

[stdout:0] 
1427238493.2388744
1427238498.2417467


This is not much better. At least it does not block this way. There's still no way to interrupt a cell.

There's another weakness that the previous example does not show:

In [36]:
%%px --noblock
test_variable = 'Hello, World.'

<AsyncResult: execute>

In [37]:
test_variable

NameError: name 'test_variable' is not defined

Let's attempt to use threading directly.

In [49]:
import threading
import time

In [50]:
run = True
def test_loop():
    i = 0
    while run:
        print(i)
        i += 1
        time.sleep(1)

In [51]:
thrd = threading.Thread(target=test_loop)

In [52]:
thrd.start()

0
1
2
3
4


In [53]:
run = False

5


Now, we'd like to be able to kill running threads.

In [54]:
import ctypes
import thread

In [61]:
class ThreadInterruptException(BaseException):
    pass

def kill_thread(thrd):
    ctypes.pythonapi.PyThreadState_SetAsyncExc(ctypes.c_long(thrd.ident),
                                               ctypes.py_object(ThreadInterruptException))

In [62]:
run = True

In [63]:
thrd = threading.Thread(target=test_loop)

In [64]:
thrd.start()

0


In [65]:
kill_thread(thrd)

1


In [66]:
'test'

2


Exception in thread Thread-8:
Traceback (most recent call last):
  File "/usr/lib/python2.7/threading.py", line 810, in __bootstrap_inner
    self.run()
  File "/usr/lib/python2.7/threading.py", line 763, in run
    self.__target(*self.__args, **self.__kwargs)
  File "<ipython-input-50-f257366e900e>", line 5, in test_loop
    print(i)
ThreadInterruptException



'test'

As you can see, this kind of works. However, the output of stdout and exceptions both end up in the wrong place.

This proof on concept works, so it should be possible to add concurrent cell execution without modifying the ipython / jupyter source code. I've also considered how to integrate concurrent execution with the current infrastructure. Unfortunately, the existing infrastructure is poorly situated for this particular task. Therefore, I'll probably write a scheduler system with custom magic from scratch.